In [5]:
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 1.0 MB/s eta 0:00:00m eta 0:00:010:01:01


In [28]:
import os 
import random 
from string import ascii_letters
import numpy as np
import torch 
from torch import nn 
import  torch.nn.functional as F 
from unidecode import unidecode

_ = torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
!ls

data  data.zip	RNN-scratch.ipynb


In [4]:
data_dir = './data/names'
lang2label = {
    file_name.split(".")[0]: torch.tensor([i], dtype=torch.long)
    for i, file_name in enumerate(os.listdir(data_dir))
}
lang2label

{'Spanish': tensor([0]),
 'Korean': tensor([1]),
 'Chinese': tensor([2]),
 'French': tensor([3]),
 'Portuguese': tensor([4]),
 'Russian': tensor([5]),
 'Czech': tensor([6]),
 'Vietnamese': tensor([7]),
 'Dutch': tensor([8]),
 'Japanese': tensor([9]),
 'Polish': tensor([10]),
 'Arabic': tensor([11]),
 'Scottish': tensor([12]),
 'German': tensor([13]),
 'English': tensor([14]),
 'Italian': tensor([15]),
 'Irish': tensor([16]),
 'Greek': tensor([17])}

## Preprocessing


In [7]:
unidecode("Ślusàrski")

'Slusarski'

In [8]:
unidecode("अंग्रेजीमा")

'aNgrejiimaa'

In [12]:
char2idx = {letter: i for i, letter in enumerate(ascii_letters + " .,:;-'")}
num_letters = len(char2idx); num_letters

59

In [13]:
def name2tensor(name):
    tensor = torch.zeros(len(name), 1, num_letters)
    for i, char in enumerate(name):
        tensor[i][0][char2idx[char]] = 1
    return tensor

In [30]:

name2tensor("abc").numpy()

array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

## Dataset Creation::
from `german.txt`


In [15]:
tensor_names = []
target_langs = []

for file in os.listdir(data_dir):
    with open(os.path.join(data_dir, file)) as f:
        lang = file.split(".")[0]
        names = [unidecode(line.rstrip()) for line in f]
        for name in names:
            try:
                tensor_names.append(name2tensor(name))
                target_langs.append(lang2label[lang])
            except KeyError:
                pass

In [26]:
tensor_names[0][0]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.]])

In [40]:
# df = [
#     [tensor_names[i] , target_langs[i]]  for i in range(len(tensor_names))
# ]
X_train , X_test , y_train , y_test = train_test_split(
    tensor_names , target_langs,test_size=0.2,random_state=42
)

In [118]:
# train_idx, test_idx = train_test_split(
#     range(len(target_langs)), 
#     test_size=0.1, 
#     shuffle=True, 
#     stratify=target_langs
# )
# (train_idx , test_idx)

In [71]:
# from sklearn.model_selection import train_test_split

# train_idx, test_idx = train_test_split(
#     range(len(target_langs)), 
#     test_size=0.1, 
#     shuffle=True, 
#     stratify=target_langs
# )

# train_dataset = [
#     (tensor_names[i], target_langs[i])
#     for i in train_idx
# ]

# test_dataset = [
#     (tensor_names[i], target_langs[i])
#     for i in test_idx
# ]
train_dataset = [(X_train , y_train)]

In [128]:
# X_train[0].numpy()

In [62]:
print(f"Train: {len(X_train)}")
print(f"Test: {len(X_test)}")

Train: 16056
Test: 4014


## Model

a simple RNN model from scratch using Pytorch 

nn.init to initialize at the hidden layers 
kaiming_uniform to initialze hidden states

In [306]:
class SimpleRNN(nn.Module):
    
    def __init__(self, input_size:int, hidden_size:int , output_size:int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        self.inp2hidden = nn.Linear(input_size , hidden_size , bias=False)
        self.hid2out = nn.Linear(hidden_size , output_size)
        
    
    def forward(self , x, hidden_state):
        """
        Returns computed output and tanh(i2h + h2h)
        Inputs
        ------
        x: Input vector
        hidden_state: Previous hidden state
        Outputs
        -------
        out: Linear output (without activation because of how pytorch works)
        hidden_state: New hidden state matrix
        """
        x = self.inp2hidden(x)
        hidden_state = torch.tanh(x+hidden_state)
        out  = self.hid2out(hidden_state)
        return out , hidden_state
    
    def init_zero_hidden(self , batch_size = 1):
        """
        Helper function.
        Returns a hidden state with specified batch size. Defaults to 1
        
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=False)
        

In [307]:
# class SimpleRNN(nn.Module):
#     def __init__(self , input_size , hidden_size , output_size):
#         super().__init__()
#         self.hidden_size = hidden_size
#         self.inp2hidden = nn.Linear(input_size+hidden_size , hidden_size)
#         self.inp2output = nn.Linear(input_size+hidden_size  ,output_size)
        
#     def forward(self , x , hidden_state):
#         print("inside forward")
#         print(x.shape)
#         print(hidden_state.shape)
#         combined = torch.cat((x,hidden_state),1 )
        
#         hidden = torch.sigmoid(self.inp2hidden(combined))
#         output = self.inp2output(combined)
        
#         return output,hidden
    
#     def init_hidden(self):
#         return nn.init.kaiming_uniform_(torch.empty(9,1 , 59))

In [308]:
hidden_size = 256
learning_rate = 0.001
num_langs = len(lang2label)
print(num_letters)
model = SimpleRNN(num_letters, hidden_size, num_langs)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

59


In [309]:
tensor_names[0].shape

torch.Size([5, 1, 59])

In [310]:
model.init_zero_hidden().shape

torch.Size([1, 256])

In [311]:
y_train[0].shape

torch.Size([1])

In [312]:
num_epochs = 2
print_interval = 3000

for epoch in range(num_epochs):
#     random.shuffle(X_train)
    for i , (name, label) in enumerate(train_dataset):
        hidden_state = model.init_zero_hidden()
        for char in name:
            print (char.shape)    
            print(hidden_state.shape)
            output, hidden_state = model(char, hidden_state)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
        if (i + 1) % print_interval == 0:
            print(
                f"Epoch [{epoch + 1}/{num_epochs}], "
                f"Step [{i + 1}/{len(train_dataset)}], "
                f"Loss: {loss.item():.4f}"
            )

torch.Size([9, 1, 59])
torch.Size([1, 256])
torch.Size([7, 1, 59])
torch.Size([9, 1, 256])


RuntimeError: The size of tensor a (7) must match the size of tensor b (9) at non-singleton dimension 0